In [45]:
import pandas as pd
import pandasql as ps
import numpy as np
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_summary import DataFrameSummary

%matplotlib inline

In [2]:
portfolio = pd.read_json('../data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/transcript.json', orient='records', lines=True)

In [3]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [19]:
profile.head(20)

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN
5,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0
6,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN
7,118,20171002,None,68617ca6246f4fbc85e91a2a49552598,NaN
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0
9,118,20161122,None,8974fc5686fe429db53ddde067b88302,NaN


年齢で年収回帰できるかと思ったが、　ageが118のところはgenderもincomeもNULLのようだ

In [4]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


income少しnullあるか

In [5]:
profile.gender.value_counts()

M    8484
F    6129
O     212
Name: gender, dtype: int64

genderもnullあるな

became_member_onが日付じゃない

In [6]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


channelが配列型  
difficulty何？ -> オファーを完了するために必要な購入金額  
あ、このデータ少ないんや

In [7]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [8]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [9]:
transcript[transcript.event == 'transaction'].head()

,event,person,time,value
12654,transaction,02c083884c7d45b39cc68e1314fec56c,0,{'amount': 0.8300000000000001}
12657,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,{'amount': 34.56}
12659,transaction,54890f68699049c2a04d415abc25e717,0,{'amount': 13.23}
12670,transaction,b2f1cd155b864803ad8334cdf13c4bd2,0,{'amount': 19.51}
12671,transaction,fe97aa22dd3e48c8b143116a8403dd52,0,{'amount': 18.97}


In [10]:
transcript[transcript.person == 'b2f1cd155b864803ad8334cdf13c4bd2']

,event,person,time,value
6657,offer received,b2f1cd155b864803ad8334cdf13c4bd2,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
12670,transaction,b2f1cd155b864803ad8334cdf13c4bd2,0,{'amount': 19.51}
27853,transaction,b2f1cd155b864803ad8334cdf13c4bd2,42,{'amount': 27.8}
34069,offer viewed,b2f1cd155b864803ad8334cdf13c4bd2,66,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
42039,transaction,b2f1cd155b864803ad8334cdf13c4bd2,102,{'amount': 17.53}
51355,transaction,b2f1cd155b864803ad8334cdf13c4bd2,156,{'amount': 20.46}
59854,offer received,b2f1cd155b864803ad8334cdf13c4bd2,168,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
79442,offer viewed,b2f1cd155b864803ad8334cdf13c4bd2,198,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
87151,transaction,b2f1cd155b864803ad8334cdf13c4bd2,222,{'amount': 27.45}
117501,offer received,b2f1cd155b864803ad8334cdf13c4bd2,336,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}


transactionは何か購入している。  
timeの意味は？ -> 開始からのhour

とりあえずvalueから`offer id`を取り出すか

In [11]:
def extract_offer_id(text):
    offer_id = re.findall(r"^{'offer id': '(.*)'}$", str(text))
    if len(offer_id) == 0:
        return np.nan
    
    return offer_id[0]

def extract_amount(text):
    offer_id = re.findall(r"^{'amount': (.*)}$", str(text))
    if len(offer_id) == 0:
        return np.nan
    
    return offer_id[0]

In [12]:
text = "{'offer id': 28.1}"
test = re.findall(r"^{'offer id': '(.*)'}$", text)
len(test)

0

In [14]:
#transcript["offer_id"] = transcript.value.map(lambda x: re.findall(r"^{'offer id': '(.*)'}$", str(x)))
transcript["offer_id"] = transcript.value.map(extract_offer_id)
#transcript["offer_id"] = transcript.value.map(lambda x: x['offer id'])
transcript["amount"] = transcript.value.map(extract_amount)

In [15]:
transcript.head()

,event,person,time,value,offer_id,amount
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},2906b810c7d4411798c6938adc9daaa5,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},fafdcd668e3743c1bb461111dcafc2a4,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


取り出せたっぽい  
残りのデータも加工していく

In [16]:
portfolio["num_channel"] = portfolio.channels.map(lambda x: len(x))

In [17]:
portfolio[portfolio.num_channel == 4]

,channels,difficulty,duration,id,offer_type,reward,num_channel
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,4
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,4
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2,4
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,4


In [18]:
channels = ['web', 'email', 'mobile', 'social']
for channel in channels:
    col_name = 'channel_' + channel
    portfolio[col_name] = portfolio.channels.map(lambda x: 1 if channel in x else 0)

portfolio.head()

,channels,difficulty,duration,id,offer_type,reward,num_channel,channel_web,channel_email,channel_mobile,channel_social
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,3,0,1,1,1
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,4,1,1,1,1
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0,3,1,1,1,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,3,1,1,1,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,2,1,1,0,0


In [26]:
transaction_transcript = transcript[transcript.event == 'transaction']
transaction_transcript = transaction_transcript.drop(['offer_id', 'value'], axis=1)

In [27]:
transaction_transcript[transaction_transcript.person == 'b2f1cd155b864803ad8334cdf13c4bd2']

,event,person,time,amount
12670,transaction,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
27853,transaction,b2f1cd155b864803ad8334cdf13c4bd2,42,27.8
42039,transaction,b2f1cd155b864803ad8334cdf13c4bd2,102,17.53
51355,transaction,b2f1cd155b864803ad8334cdf13c4bd2,156,20.46
87151,transaction,b2f1cd155b864803ad8334cdf13c4bd2,222,27.45
143588,transaction,b2f1cd155b864803ad8334cdf13c4bd2,384,29.95
191243,transaction,b2f1cd155b864803ad8334cdf13c4bd2,468,24.55
243244,transaction,b2f1cd155b864803ad8334cdf13c4bd2,570,28.1


In [31]:
offer_receive_transcript = transcript[transcript.event == 'offer received']
offer_receive_transcript = offer_receive_transcript.drop(['event', 'value', 'amount'], axis=1)

In [34]:
offer_receive_transcript = offer_receive_transcript.merge(portfolio[['id', 'duration']], how='left', left_on='offer_id', right_on='id').drop('id', axis=1)

In [46]:
offer_receive_transcript.head()

,person,time,offer_id,duration,next_time
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,0.0
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,7,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,10,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,5,0.0


In [60]:
query = '''
    SELECT
        person,
        time AS offer_received_at,
        offer_id,
        duration,
        LEAD(time, 1, 600) OVER(PARTITION BY person ORDER BY time ASC) AS next_offer_send_at
    FROM
      offer_receive_transcript
'''

offer_receive_transcript_step1 = ps.sqldf(query, locals())

In [61]:
transaction_transcript_step2 = transaction_transcript.merge(offer_receive_transcript_step1, how='left', on='person')
transaction_transcript_step2.head()

,event,person,time,amount,offer_received_at,offer_id,duration,next_offer_send_at
0,transaction,02c083884c7d45b39cc68e1314fec56c,0,0.8300000000000001,0.0,ae264e3637204a6fb9bb56bc8210ddfd,7.0,408.0
1,transaction,02c083884c7d45b39cc68e1314fec56c,0,0.8300000000000001,408.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10.0,600.0
2,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,0.0,2906b810c7d4411798c6938adc9daaa5,7.0,336.0
3,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,7.0,408.0
4,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,408.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10.0,504.0


In [67]:
transaction_transcript_step2[transaction_transcript_step2.person == '9fa9ae8f57894cc9a3b8a9bbe0fc1b2f']

,event,person,time,amount,offer_received_at,offer_id,duration,next_offer_send_at
2,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,0.0,2906b810c7d4411798c6938adc9daaa5,7.0,336.0
3,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,7.0,408.0
4,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,408.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10.0,504.0
5,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,504.0,4d5c57ea9a6940dd891ad53e9dbe8da0,5.0,600.0
28279,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,42,21.55,0.0,2906b810c7d4411798c6938adc9daaa5,7.0,336.0
28280,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,42,21.55,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,7.0,408.0
28281,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,42,21.55,408.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10.0,504.0
28282,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,42,21.55,504.0,4d5c57ea9a6940dd891ad53e9dbe8da0,5.0,600.0
84383,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,114,32.87,0.0,2906b810c7d4411798c6938adc9daaa5,7.0,336.0
84384,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,114,32.87,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,7.0,408.0


In [74]:
# 購入よりも後のオファーに対するレコードを除去
transaction_transcript_step3 = transaction_transcript_step2[(
    transaction_transcript_step2.time.astype(float) >= transaction_transcript_step2.offer_received_at) &
    (transaction_transcript_step2.time.astype(float) < transaction_transcript_step2.next_offer_send_at
)]

In [71]:
transaction_transcript_step3[transaction_transcript_step3.person == 'b2f1cd155b864803ad8334cdf13c4bd2']

,event,person,time,amount,offer_received_at,offer_id,duration,next_offer_send_at
9,transaction,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,0.0,5a8bc65990b245e5a138643cd4eb9837,3.0,168.0
28293,transaction,b2f1cd155b864803ad8334cdf13c4bd2,42,27.8,0.0,5a8bc65990b245e5a138643cd4eb9837,3.0,168.0
75891,transaction,b2f1cd155b864803ad8334cdf13c4bd2,102,17.53,0.0,5a8bc65990b245e5a138643cd4eb9837,3.0,168.0
112055,transaction,b2f1cd155b864803ad8334cdf13c4bd2,156,20.46,0.0,5a8bc65990b245e5a138643cd4eb9837,3.0,168.0
166381,transaction,b2f1cd155b864803ad8334cdf13c4bd2,222,27.45,168.0,3f207df678b143eea3cee63160fa8bed,4.0,336.0
298244,transaction,b2f1cd155b864803ad8334cdf13c4bd2,384,29.95,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,7.0,408.0
387209,transaction,b2f1cd155b864803ad8334cdf13c4bd2,468,24.55,408.0,fafdcd668e3743c1bb461111dcafc2a4,10.0,504.0
492317,transaction,b2f1cd155b864803ad8334cdf13c4bd2,570,28.1,504.0,ae264e3637204a6fb9bb56bc8210ddfd,7.0,600.0


In [75]:
transaction_transcript_step3 = transaction_transcript_step3.drop(["next_offer_send_at"], axis=1)

In [76]:
transaction_transcript_step3[transaction_transcript_step3.person == 'b2f1cd155b864803ad8334cdf13c4bd2']

,event,person,time,amount,offer_id,duration
9,transaction,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,5a8bc65990b245e5a138643cd4eb9837,3.0
28293,transaction,b2f1cd155b864803ad8334cdf13c4bd2,42,27.8,5a8bc65990b245e5a138643cd4eb9837,3.0
75891,transaction,b2f1cd155b864803ad8334cdf13c4bd2,102,17.53,5a8bc65990b245e5a138643cd4eb9837,3.0
112055,transaction,b2f1cd155b864803ad8334cdf13c4bd2,156,20.46,5a8bc65990b245e5a138643cd4eb9837,3.0
166381,transaction,b2f1cd155b864803ad8334cdf13c4bd2,222,27.45,3f207df678b143eea3cee63160fa8bed,4.0
298244,transaction,b2f1cd155b864803ad8334cdf13c4bd2,384,29.95,2298d6c36e964ae4a3e7e9706d1fb8c2,7.0
387209,transaction,b2f1cd155b864803ad8334cdf13c4bd2,468,24.55,fafdcd668e3743c1bb461111dcafc2a4,10.0
492317,transaction,b2f1cd155b864803ad8334cdf13c4bd2,570,28.1,ae264e3637204a6fb9bb56bc8210ddfd,7.0


In [ ]:
offer_viewed_transcript = transcript[transcript.event == 'offer viewed']

query = '''
    SELECT
        person,
        time AS offer_viewed_at,
        offer_id,
        duration,
        LEAD(time, 1, 600) OVER(PARTITION BY person ORDER BY time ASC) AS next_offer_send_at
    FROM
      offer_viewed_transcript
'''